<a href="https://colab.research.google.com/github/Tinynja/Sarsa-phi-EB/blob/main/ALE_Framework_Tests.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

if 'google.colab' in sys.modules:
    !rm -rf *
    !git clone https://github.com/Tinynja/Sarsa-phi-EB
    !mv Sarsa-phi-EB/* .
    !rm -rf Sarsa-phi-EB
    # DON'T install packages defined in Pipfile_colab_remove
    !sed -ri "/$(tr '\n' '|' < Pipfile_colab_remove)/d" Pipfile
else:
    print('Skipping GitHub cloning since not running in Colab.')

Cloning into 'Sarsa-phi-EB'...
remote: Enumerating objects: 167, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (143/143), done.
remote: Total 167 (delta 24), reused 146 (delta 16), pack-reused 0
Receiving objects: 100% (167/167), 504.44 KiB | 2.02 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [2]:
# Install required dependencies
if 'google.colab' in sys.modules:
    # Colab doesn't support pipenv, hence we convert Pipfile into requirements.txt
    !pip install pipenv 1> /dev/null
    !pipenv lock -r > requirements.txt
    !pip install -r requirements.txt 1> /dev/null
else:
    !pipenv lock 1> /dev/null
    !pipenv install --deploy 1> /dev/null

Creating a virtualenv for this project...
Pipfile: /content/Pipfile
Using /usr/local/bin/python (3.7.12) to create virtualenv...
⠼ Creating virtual environment...created virtual environment CPython3.7.12.final.0-64 in 908ms
  creator CPython3Posix(dest=/root/.local/share/virtualenvs/content-cQIIIOO2, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==21.3.1, setuptools==58.3.0, wheel==0.37.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator

✔ Successfully created virtual environment! 
Virtualenv location: /root/.local/share/virtualenvs/content-cQIIIOO2
Pipfile.lock not found, creating...
Locking [dev-packages] dependencies...
Locking [packages] dependencies...
Building requirements...
Resolving dependencies...
✔ Success! 
Updated Pipfile.lock (44ce9b)!
ERROR: pip's 

In [3]:
# Import all supported ROMs into ALE
!ale-import-roms ROMS

[SUPPORTED]                    qbert        ROMS/Q. Bert (1983).bin
[SUPPORTED]                   casino ROMS/Casino - Poker Plus (Paddle) (1979).bin
[SUPPORTED]             demon_attack ROMS/Demon Attack (Death from Above) (1982).bin
[SUPPORTED]                  bowling        ROMS/Bowling (1979).bin
[SUPPORTED]                 seaquest       ROMS/Seaquest (1983).bin
[SUPPORTED]          chopper_command ROMS/Chopper Command (1982).bin
[SUPPORTED]               time_pilot     ROMS/Time Pilot (1983).bin
[SUPPORTED]                 trondead ROMS/TRON - Deadly Discs (TRON Joystick) (1983).bin
[SUPPORTED]                jamesbond ROMS/James Bond 007 (James Bond Agent 007) (1984).bin
[SUPPORTED]                   pacman        ROMS/Pac-Man (1982).bin
[SUPPORTED]                   kaboom ROMS/Kaboom! (Paddle) (1981).bin
[SUPPORTED]                  asterix ROMS/Asterix (AKA Taz) (1983).bin
[SUPPORTED]                    qbert         ROMS/Q-bert (1987).bin
[SUPPORTED]                  pitfal

In [4]:


# Check if we can display the screen
# print(SDL_SUPPORT)
# if SDL_SUPPORT:
#     ale.setBool("sound", True)
#     ale.setBool("display_screen", True)
# else:
#     ale.setBool("sound", False)
#     ale.setBool("display_screen", False)


# Get the list of legal actions


In [5]:
# Built-in libraries
import sys
import random

# Pipy libraries
from ale_py import ALEInterface, SDL_SUPPORT
from ale_py.roms import Breakout

/usr/local/lib/python3.7/dist-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [6]:
# Setup
ale = ALEInterface()
ale.setInt("random_seed", 0)

if SDL_SUPPORT and 'google.colab' not in sys.modules:
    ale.setBool("sound", True)
    ale.setBool("display_screen", True)

ale.loadROM(Breakout)
legal_actions = ale.getLegalActionSet()

# Play 10 episodes
for episode in range(10):
    total_reward = 0
    while not ale.game_over():
        action = random.choice(legal_actions)
        # Apply an action and get the resulting reward
        reward = ale.act(action)
        total_reward += reward
    print("Episode %d ended with score: %d" % (episode, total_reward))
    ale.reset_game()

Episode 0 ended with score: 2
Episode 1 ended with score: 4
Episode 2 ended with score: 3
Episode 3 ended with score: 3
Episode 4 ended with score: 0
Episode 5 ended with score: 0
Episode 6 ended with score: 1
Episode 7 ended with score: 2
Episode 8 ended with score: 1
Episode 9 ended with score: 1
